<a href="https://colab.research.google.com/github/tutsilianna/Tools_and_Methods_of_math_tech/blob/main/lab2/%D0%98%D0%B8%D0%9C%D0%9C%D0%A2_%D0%9B%D0%A0_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Лабораторная работа №2**


---


**Команда №2:**

*Давыдова Кристина Сергеевна*        
*Поздышева Татьяна Сергеевна*

---



**"Возведение чисел в степень по модулю"**

*Цель лабораторной работы*: используя результаты теории чисел, реализовать возведение числа в степень по модулю. Данные преобразования используются в криптографии, например, в алгоритме RSA.

**Постановка задачи:**

Найти результат преобразования: $$ c \equiv a^b \mod \; M $$

1. Факторизовать число $M$, по модулю которого производится возведение в степень.
2. Найти вектор остатков {$r_i$} по модулям простых чисел {$a_i$} таких, что $M$ = $∏_{i=1}^{k} a_i$

3. Используя малую теорему Ферма найти $\tilde{r}_i \equiv r^b_i \; mod \; a_i$
4. Реализовать восстановление числа $c$ по его остаткам $\{\tilde{r}_i\}$ при помощи Китайской теоремы об остатках.

## Подключение библиотек

In [ ]:
import numpy as np

from functools import reduce
import math

In [ ]:
from sympy import pprint    # для вывода делителей числа и их степени
from sympy import factorint # для факторизации числа

## Исходные данные

In [ ]:
a = 3
b = 4
M = 10

In [ ]:
# другой вариант исходных данных
A = 15
B = 4
m = 77

## Факторизуем число M, по модулю которого производится возведение в степень

Для факторизации числа M (в данном случае 10), используется функция `factorize()`. Эта функция находит все простые делители числа M и возвращает их в виде списка.

In [ ]:
def factorize(M):
    factors = []
    i = 2
    while i <= M:
        if M % i == 0:
            factors.append(i)
            M //= i
        else:
            i += 1
    return factors

Затем переменная `c` получает значение `a` в степени `b` по модулю числа M, то есть $c = a^b \mod \; M$ (в данном случае $3^4 \mod\;10$).

На экран выводятся результат факторизации (список простых делителей) числа $M$ и значение переменной $c$.

In [ ]:
factors_of_M = factorize(M)
c = pow(a, b, M)

print(f"{a} в степени {b} по модулю {M} равно {c}")
print("Простые множители числа M:", factors_of_M)
print()
# для других исходных данных для сравнения результата:
factors_of_m = factorize(m)
C = pow(A, B, m)

print(f"{A} в степени {B} по модулю {m} равно {C}")
print("Простые множители числа M:", factors_of_m)

3 в степени 4 по модулю 10 равно 1
Простые множители числа M: [2, 5]

15 в степени 4 по модулю 77 равно 36
Простые множители числа M: [7, 11]


### **Второй вариант факторизации числа с помощью библиотеки SymPy**

In [ ]:
factors_M = factorint(M, multiple=True)       # это и есть a_i

print(f"Факторы числа M = {M}: {', '.join(map(str, factors_M))}")
print()
pprint(factorint(M, visual=True))

print()

factors_m = factorint(m, multiple=True)       # это и есть a_i

print(f"Факторы числа M = {m}: {', '.join(map(str, factors_m))}")
print()
pprint(factorint(m, visual=True))

Факторы числа M = 10: 2, 5

 1  1
2 ⋅5 

Факторы числа M = 77: 7, 11

  1  1
11 ⋅7 


## Вектор остатков по модулям простых чисел

После вычисления значения переменной $c$, вектор остатков $r$ находится с помощью цикла `for`. В каждой итерации цикла значение `r_i` равно $a$ в степени $b$, по модулю текущего простого делителя $a_i$: $r_i = a^b \mod \; a_i$, а затем оно добавляется в вектор `r` при помощи метода `append()`.

В результате, выводится на экран вектор остатков `r`.

In [ ]:
r = []
for ai in factors_of_M:
    r_i = pow(a, b, ai)
    r.append(r_i)

print(f"Вектор остатков r: {r} для числа M={M} и чисел a={a}, b={b}")

Вектор остатков r: [1, 1] для числа M=10 и чисел a=3, b=4


In [ ]:
R = []
for ai in factors_of_m:
    r_i = pow(A, B, ai)
    R.append(r_i)

print(f"Вектор остатков r: {R} для числа M={m} и чисел a={A}, b={B}")

Вектор остатков r: [1, 3] для числа M=77 и чисел a=15, b=4


## Использование малой теоремы Ферма

Мы можем использовать малую теорему Ферма для нахождения остатков числа $M$ по модулям простых чисел.

**Малая теорема Ферма**:
если $p$ - простое число и $a$ - целое число, которое не делится на $p$, то $a^{p-1} ≡ 1$, где $≡$ обозначает сравнение по модулю $p$. Другими словами, $a^{m-1}-1$ делится на $p$.

Используя данную теорему, мы можем найти остаток числа $M$ по модулю простого числа $p$, вычислив $M^{p-1} \mod p$.

Новая функция `apply_fermat_theorem()`, которая использует малую теорему Ферма для вычисления остатка `r_i1`. В этой функции `pow(a, b % (ai - 1), ai)` вычисляет значение $a^{b \mod (a_i - 1)}$, по модулю $a_i$.

Затем, в цикле `for` для каждого простого делителя `ai` числа `M`, вычисляется остаток `r_i1` с помощью функции `apply_fermat_theorem()`, и добавляется в вектор `r_1`.

Таким образом, в результате программы на экран будет выведен вектор остатков `r`, вычисленных с использованием малой теоремы Ферма.

In [ ]:
def apply_fermat_theorem(a, b, ai):
    return pow(a, b % (ai - 1), ai)

r_1 = []
for ai in factors_of_M:
    r_i1 = apply_fermat_theorem(a, b, ai)
    r_1.append(r_i1)

print(f"r по малой теореме Ферма для числа M={M} и чисел a={a}, b={b}: {', '.join(map(str, r_1))}")

r по малой теореме Ферма для числа M=10 и чисел a=3, b=4: 1, 1


In [ ]:
R_1 = []
for ai in factors_of_m:
    R_1.append(apply_fermat_theorem(A, B, ai))

print(f"r по теореме Ферма для числа M={m} и чисел a={A}, b={B}: {', '.join(map(str, R_1))}")

r по теореме Ферма для числа M=77 и чисел a=15, b=4: 1, 3


### Используя малую теорему Ферма, найдем $\tilde{r}_i \equiv r^b_i \; mod \; a_i$

In [ ]:
r_1_ = []
for i in range(len(factors_of_M)):
    r_1_.append(pow(r_1[i], b, factors_of_M[i]))
r_1_

[1, 1]

In [ ]:
R_1_ = []
for i in range(len(factors_of_m)):
    R_1_.append(pow(R_1[i], B, factors_of_m[i]))
R_1_

[1, 4]

##Восстановление числа $c$ по его остаткам при помощи Китайской теоремы об остатках

Мы можем реализовать восстановление числа $M$ по его остаткам с использованием Китайской теоремы об остатках.

**Китайская теорема об остатках** утверждает следующее:

Для любых целых чисел $a_1, a_2, ..., a_n$ и положительных целых чисел $m_1, m_2, ..., m_n$, если все числа $m_i$ попарно взаимно просты, то существует целое число $x$, которое удовлетворяет системе сравнений:

$x ≡ a_1 (\mod \; m_1)$

$x ≡ a_2 (\mod \; m_2)$

...

$x ≡ a_n (\mod \; m_n)$

Добавляются две новые функции: `chinese_remainder_theorem()` и `mul_inv()`.

Функция `chinese_remainder_theorem(n, a)` реализует Китайскую теорему об остатках. Она принимает два массива `ai` и `ri`, где `ai` - это список простых делителей числа `M`, а `ri` - это список остатков `r_i`. Функция вычисляет восстановленное значение числа `c` на основе остатков и возвращает его.

Функция `mul_inv(a, b)` реализует нахождение мультипликативного обратного числа `a` по модулю `b` при помощи алгоритма расширенного алгоритма Евклида.

In [ ]:
def mul_inv(a, b):
    b0 = b
    x0, x1 = 0, 1
    if b == 1:
        return 1
    while a > 1:
        q = a // b
        a, b = b, a % b
        x0, x1 = x1 - q * x0, x0
    if x1 < 0:
        x1 += b0
    return x1

def chinese_remainder_theorem(ai, ri): # M = П a_i, r = {r_i}
    sum_ = 0
    prod = reduce(lambda x, y: x * y, ai) # произведение a_i = M
    for a_i, r_i in zip(ai, ri):
        p = prod // a_i
        sum_ += r_i * mul_inv(p, a_i) * p
    return sum_ % prod

# print(chinese_remainder_theorem([3,5,7], [2,3,2]))

23


В результате, на экран будет выведено восстановленное значение числа `c` при помощи Китайской теоремы об остатках.

In [ ]:
restored_c = chinese_remainder_theorem(factors_of_M, r)

print(f"Восстановленное число c для числа M={M} и чисел a={a}, b={b}: {restored_c}")

Восстановленное число c для числа M=10 и чисел a=3, b=4: 1


In [ ]:
restored_C = chinese_remainder_theorem(factors_of_m, R)

print(f"Восстановленное число c для числа M={m} и чисел a={A}, b={B}: {restored_C}")

Восстановленное число c для числа M=77 и чисел a=15, b=4: 36
